In [ ]:
!uv pip install "/Users/shiv/Documents/GitHub/notte" # local version
print(notte.__file__)
print(notte.__version__)

In [ ]:
import os
import sys
from loguru import logger
from dotenv import load_dotenv
load_dotenv()

import notte
from notte_sdk import NotteClient
from notte_sdk.types import ExternalProxy
from notte_core.actions import FormFillAction
from options import iterate_options, SDKSessionOptions, AgentOptions, SessionOptions

In [4]:
client = NotteClient(api_key=os.environ["NOTTE_API_KEY"], server_url="http://localhost:8000")
proxies=[ ExternalProxy(server="https://isp.oxylabs.io:8001", username="dedicated_4iJWG", password="Zlf1daSj13~An5a+")]

WARNING  - NOTTE_API_URL is set to: http://localhost:8000


In [5]:
form_fill_action =FormFillAction(
    value={
    'first_name': 'Lucas',
    'last_name': 'Harmsell',
    #'full_name': 'Lucas Harmsell',
    'email': 'lucas@notte.cc',
    'phone': '+4917688888888',
    'address1': '730 Clayton St',
    'city': 'San Francisco',
    'state': 'CA',
    'country': 'United States',
    'postal_code': "94117",
    }
)

zip_fill_action =FormFillAction(
    value={
    'postal_code': "94117",
    }
)

# sequence = [
#     {"type": "goto", "value": "https://www.teepublic.com/t-shirt/77306245-we-should-have-more-billionaires"},
#     {"type": "click", "action_id": "B1"},
#     {"type": "click", "action_id": "M21"},
#     {"type": "click", "action_id": "B26"},
#     {"type": "click", "action_id": "L19"},
#     {"action": form_fill_action},
#     {"type": "scroll_down"},
#     {"action": zip_fill_action},
#     {"type": "click", "action_id": "B1"},
#     {"type": "click", "action_id": "B1"},
# ]


# sys.path.append('/Users/shiv/Documents/GitHub')
# import notte


# vault = client.Vault("a5cce28b-6fa2-418e-ac04-23b3dc9102b0")
d = SessionOptions()
# del d['browser_type']
session = notte.Session(proxies=proxies)
session.start()

#with notte.Session(browser_type="chrome", proxies=False,  viewport_width=1280, viewport_height=1080) as session:
# for action in sequence:
#     res = session.execute(**action)
#     logger.info(f"Success: {res.success} - {res.message}")
#     obs = session.observe(perception_type="fast")
#     logger.info(obs)
        
    

url = "https://www.teepublic.com/t-shirt/77306245-we-should-have-more-billionaires"
agent = notte.Agent(
    session=session,
    #vault=vault,
    use_vision=False,
    # reasoning_model="vertex_ai/gemini-2.0-flash"
    reasoning_model="cerebras/llama-3.3-70b",
    max_steps=1,
)
# Note: you may need to close a popup message from the company to access the payment button.
response = agent.run(
    task=f"""\
1. go to {url} and close any modal/popup that appears
2. Select size `L` for the T-shirt
3. click on the add to cart button
4. click on the checkout button
5. fill in the form with the following information (use the `form_fill` action to fill all fields at once): {form_fill_action.value}. 
    CRITICAL: once the form action is executed, your next actions should always be to scroll down.
    CRITICAL: After scrolling down, check if the postal code is filled correctly
    CRITICAL: you are guaranteed that all fields are filled (except for the postal code). Don't try to refill the form elements, except for the postal code.
6. click on the "Continue to Payment" button (never use the PayPal or Google Pay options).
7. On the modal that appears, click on the "Use Recommended Address" button
9. Fill credit card information using the `form_fill` action

use the following credit card information:

card_holder_name="Andrea Pinto",
card_number="4165989636576537",
card_full_expiration="07/30",
card_cvv="211",


10. Click on the "Place Order" button
""",
    url=url,
        
)

WARNING  - No perception model set, using reasoning model: vertex_ai/gemini-2.0-flash
WARNING  - 🪟 No viewport set in headful mode, using default viewport in playwright
INFO     - Session url is 'about:blank': returning empty observation. Perform goto action before observing to get a more meaningful observation.
ERROR    - NotteBaseError: Failed to load page from the given URL. Check if the URL is reachable.
INFO     - 🚨 Failed to solve task in 1 steps
/Users/shiv/Documents/GitHub/demos/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# PROMPT = """\
# Look up the English word "computer" and identify its French translation, \
# noting any pronunciation or image resources displayed. Only use https://glosbe.com/ to \
# achieve the task. Don't go to any other site. The task is achievable with just navigation from this site.\
# """


# {
#     'vault' : NotteClient().Vault(),

#     # Add your credentials securely
#     '_' : vault.add_credentials(
#         url="https://github.com/",
#         email="<your-email>",
#         password="<your-password>",
#         mfa_secret="<your-mfa-secret>",
#     )
# }


# # output_filename = 'NVIDIA_SEC_FILING.csv'
# # output_dir = 'NVIDIA_SEC_FILING/'
# # storage = LocalStorage(download_dir=output_dir)

# # option_rank = [
# #     1
# #     2,
# #     function to make rest full search
# # ]

# client = NotteClient()

# with client.Session(**client_session_options) as session:
#     agent = client.Agent(session=session, **agent_options)
#     response = agent.run(task=PROMPT)
#     print(response)


In [29]:
response.trajectory[0].space.actions


[FormFillAction(type='form_fill', category='Special Browser Actions', description='Fill a form with multiple values. Critical: If you detect a form on a page, try to use this action at first, and otherwise use the regular fill action', value={'email': 'hello@example.com', 'first_name': 'Johnny', 'last_name': 'Smith'}),
 GotoAction(type='goto', category='Special Browser Actions', description='Goto to a URL (in current tab)', url='<some_url>'),
 GotoNewTabAction(type='goto_new_tab', category='Special Browser Actions', description='Goto to a URL (in new tab)', url='<some_url>'),
 SwitchTabAction(type='switch_tab', category='Special Browser Actions', description='Switch to a tab (identified by its index)', tab_index=0),
 GoBackAction(type='go_back', category='Special Browser Actions', description='Go back to the previous page (in current tab)'),
 GoForwardAction(type='go_forward', category='Special Browser Actions', description='Go forward to the next page (in current tab)'),
 ReloadAction

In [32]:
actions = response.trajectory[0].space.actions
[session.execute(action=action) for action in actions]

KeyboardInterrupt: 

In [33]:
session.stop()

ERROR    - Failed to close window: Browser.close: Connection closed while reading from the driver


In [ ]:
save_dir = "setups_search"
client = NotteClient()
for SessionOptions, AgentOptions in iterate_options(local=False):
    with client.Session(**SessionOptions) as session:
        agent = notte.Agent(session=session, **AgentOptions)
        agent.run(task=prompt)
        agent.replay(screenshot_type="last_action").save(f"{save_dir}/agent.webp")